In [2]:
from pyspark.sql import functions as f
from pyspark.sql import SparkSession
from pyspark.sql.types import *

Tạo spark Session

In [6]:


jar_list = [
    r"C:\Users\dinha\OneDrive\Documents\FunixLab\cuorse3\ASM1\jars\org.mongodb_bson-4.0.5.jar",
    r"C:\Users\dinha\OneDrive\Documents\FunixLab\cuorse3\ASM1\jars\org.mongodb_mongodb-driver-core-4.0.5.jar",
    r"C:\Users\dinha\OneDrive\Documents\FunixLab\cuorse3\ASM1\jars\org.mongodb_mongodb-driver-sync-4.0.5.jar",
    r"C:\Users\dinha\OneDrive\Documents\FunixLab\cuorse3\ASM1\jars\org.mongodb.spark_mongo-spark-connector_2.12-3.0.1.jar"
]

spark = SparkSession.builder \
    .master("local") \
    .appName("ASM1_Spark") \
    .config("spark.jars", ",".join(jar_list)) \
    .config("spark.local.dir", r"C:\Users\dinha\OneDrive\Documents\FunixLab\cuorse3\ASM1\temp") \
    .config("spark.cleaner.referenceTracking", "true") \
    .getOrCreate()

Load dữ liệu từ source

In [7]:
Questition_schema = StructType([
    StructField("Body",StringType()),
    StructField("CreationDate",StringType()),
    StructField("ClosedDate",StringType()),
    StructField("Id",IntegerType()),
    StructField("OwnerUserId", StringType()),
    StructField("title",StringType()),
    StructField("Score",IntegerType()),
])

Question_df = spark.read \
    .format("com.mongodb.spark.sql.DefaultSource") \
    .option("uri","mongodb://localhost:27017/ASM1_spark") \
    .option("collection","Questitions") \
    .schema(Questition_schema) \
    .load()

In [8]:
Question_df.printSchema()

root
 |-- Body: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- ClosedDate: string (nullable = true)
 |-- Id: integer (nullable = true)
 |-- OwnerUserId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- Score: integer (nullable = true)



In [9]:
Question_df.show()

+--------------------+--------------------+--------------------+----+-----------+-----+-----+
|                Body|        CreationDate|          ClosedDate|  Id|OwnerUserId|title|Score|
+--------------------+--------------------+--------------------+----+-----------+-----+-----+
|<p>Are there any ...|2008-08-01T14:41:24Z|2012-12-26T03:45:49Z|  90|         58| NULL|  144|
|<p>I've written a...|2008-08-01T13:57:07Z|                  NA|  80|         26| NULL|   26|
|<p>This is someth...|2008-08-01T18:42:19Z|                  NA| 180|    2089740| NULL|   53|
|<p>I have a littl...|2008-08-01T23:22:08Z|                  NA| 260|         91| NULL|   49|
|<p>I am working o...|2008-08-02T02:51:36Z|                  NA| 330|         63| NULL|   29|
|<p>I've been writ...|2008-08-02T15:11:47Z|2016-03-26T05:23:29Z| 470|         71| NULL|   13|
|<p>Has anyone got...|2008-08-01T15:50:08Z|                  NA| 120|         83| NULL|   21|
|<p>I'm trying to ...|2008-08-03T20:35:01Z|                 

Chuẩn hóa dữ liệu

In [10]:
Questition_converted_schema = Question_df.withColumn("OwnerUserId",f.expr("case when OwnerUserId = 'NA' then null else OwnerUserId end").cast(IntegerType()))
Questition_converted_schema.printSchema()
Questition_converted_schema.show()

root
 |-- Body: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- ClosedDate: string (nullable = true)
 |-- Id: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Score: integer (nullable = true)

+--------------------+--------------------+--------------------+----+-----------+-----+-----+
|                Body|        CreationDate|          ClosedDate|  Id|OwnerUserId|title|Score|
+--------------------+--------------------+--------------------+----+-----------+-----+-----+
|<p>Are there any ...|2008-08-01T14:41:24Z|2012-12-26T03:45:49Z|  90|         58| NULL|  144|
|<p>I've written a...|2008-08-01T13:57:07Z|                  NA|  80|         26| NULL|   26|
|<p>This is someth...|2008-08-01T18:42:19Z|                  NA| 180|    2089740| NULL|   53|
|<p>I have a littl...|2008-08-01T23:22:08Z|                  NA| 260|         91| NULL|   49|
|<p>I am working o...|2008-08-02T02:51:36Z|                 

Yêu cầu 1: Tính số lần xuất hiện của các ngôn ngữ lập trình
+-------------------+------+                                                    
|Programing Language| Count|
+-------------------+------+
|                 C#| 32414|
|                C++| 28866|
|                CSS| 33556|
|               HTML| 89646|
|                PHP| 63479|
|                SQL|146094|
|                 Go| 79912|
|               Ruby| 16318|
|             Python| 44817|
|               Java|106659|
+-------------------+------+

Cách xử lý thứ nhất:

In [ ]:
count_word_per_record_df = Questition_converted_schema.withColumn("Body",f.lower("Body")) \
    .select(
    f.col("Id"),
    f.col("Body"), 
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])c#(?![a-zA-Z])")).alias(r"c#"),
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])c++(?![a-zA-Z])")).alias(r"c++"),
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])css(?![a-zA-Z])")).alias(r"css"),
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])html(?![a-zA-Z])")).alias(r"html"),
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])php(?![a-zA-Z])")).alias(r"php"),
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])sql(?![a-zA-Z])")).alias(r"sql"),
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])go(?![a-zA-Z])")).alias(r"go"),
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])ruby(?![a-zA-Z])")).alias(r"ruby"),
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])python(?![a-zA-Z])")).alias(r"python"),
    f.regexp_count("Body" , f.lit(r"(?<![a-zA-Z])java(?![a-zA-Z])")).alias(r"java")
    )

total_count_df = count_word_per_record_df.agg(
    f.sum("c#").alias("C#"),
    f.sum("c++").alias("c++"),
    f.sum("css").alias("css"),
    f.sum("html").alias("html"),
    f.sum("php").alias("php"),
    f.sum("sql").alias("sql"),
    f.sum("go").alias("go"),
    f.sum("ruby").alias("ruby"),
    f.sum("python").alias("python"),
    f.sum("java").alias("java")
)

In [10]:
total_count_df.show()

+-----+------+------+------+------+------+-----+-----+------+------+
|   C#|   c++|   css|  html|   php|   sql|   go| ruby|python|  java|
+-----+------+------+------+------+------+-----+-----+------+------+
|40439|672255|131181|385503|328627|119089|81979|68293|115873|722406|
+-----+------+------+------+------+------+-----+-----+------+------+



Cách xử lý thứ 2:

In [11]:
languages = ["Java", "Python", "C\+\+", "C#", "Go", "Ruby", "Javascript", "PHP", "HTML", "CSS", "SQL"]

# filter_programing_languages = Questition_converted_schema.select(f.explode(f.split(f.col("Body")," ")).alias("word")) \
#     .withColumn("word", f.lower("word")) \
#     .filter(f.col("word").rlike("|".join([f"\\b{lang.lower()}\\b" for lang in languages])))

filter_programing_languages = Questition_converted_schema.withColumn("word",f.lower("Body")) \

extracted_df = filter_programing_languages.select(f.regexp_extract_all("word", f.lit("(" + "|".join([fr"(?<![a-zA-Z]){lang.lower()}(?![a-zA-Z])" for lang in languages]) + ")"), 0).alias("language"))


sum_programing_language_df = extracted_df.select(f.explode("language").alias("Programing language")) \
    .withColumn("code", f.expr('''case when `Programing language` = 'c++' then 1
                                when `Programing language` = 'c#' then 2
                                when `Programing language` = 'css' then 3
                                when `Programing language` = 'html' then 4
                                when `Programing language` = 'php' then 5
                                when `Programing language` = 'sql' then 6
                                when `Programing language` = 'go' then 7
                                when `Programing language` = 'ruby' then 8
                                when `Programing language` = 'python' then 9
                                when `Programing language` = 'java' then 10
                                end''')) \
    .groupBy("code").agg(f.count("code").alias("count")) \
    

sum_programing_language_df.select(f.expr('''case when `code` = 1 then 'c++'
                                when `code` = 2 then 'c#'
                                when `code` = 3 then 'css'
                                when `code` = 4 then 'html'
                                when `code` = 5 then 'php' 
                                when `code` = 6 then 'sql'
                                when `code` = 7 then 'go'
                                when `code` = 8 then 'ruby' 
                                when `code` = 9 then 'python' 
                                when `code` = 10 then'java'
                                end''').alias("Programing language"), f.col("count")).show()

+-------------------+------+
|Programing language| count|
+-------------------+------+
|               NULL|     0|
|                c++| 39554|
|                sql|119089|
|                css|131181|
|                php|328627|
|             python|115873|
|               html|385503|
|               ruby| 68293|
|                 go| 81979|
|               java|722406|
|                 c#| 40439|
+-------------------+------+



Yêu cầu 2: Tìm các domain được sử dụng nhiều nhất trong các câu hỏi
Trong các câu hỏi thường chúng ta sẽ dẫn link từ các trang web khác vào. Ở yêu cầu này, bạn sẽ cần tìm xem 20 domain nào được người dùng sử dụng nhiều nhất. Chú ý rằng các domain sẽ chỉ gồm tên domain, các bạn sẽ không cần trích xuất những tham số phía sau. Ví dụ về một domain: www.google.com, www.facebook.com,...

Để hoàn thành được yêu cầu này, bạn có thể sử dụng regex để trích xuất các url, sau đó áp dung một số biện pháp xử lý chuỗi để lấy ra được tên của domain, cuối cùng là dùng Aggregation để gộp nhóm lại. Kết quả sẽ như sau:

+--------------------+-----+                                                 
|              Domain|Count|
+--------------------+-----+

|   www.cs.bham.ac.uk|    4|
|groups.csail.mit.edu|    7|
|     fiddlertool.com|    1|
|   www.dynagraph.org|    1|
| images.mydomain.com|    1|
|  img7.imageshack.us|    3|
+--------------------+-----+

In [74]:
regex_pattern = r"https?://([\w.-]+)/"

extracted_link_df = Questition_converted_schema.select(f.regexp_extract_all("Body", f.lit(regex_pattern)).alias("link_extracted")).filter(f.array_size("link_extracted") > 1)

extracted_link_df.select(f.explode(f.col("link_extracted")).alias("explode_link"),f.lit(1).alias("1")) \
    .groupBy("explode_link") \
    .agg(f.count(f.col("1"))).show()



+--------------------+--------+
|        explode_link|count(1)|
+--------------------+--------+
|   www.cs.bham.ac.uk|       3|
|     fiddlertool.com|       1|
|        www.faqs.org|      15|
| support.mozilla.com|       3|
|   www.w3schools.com|     756|
|georgia.ubuntufor...|       2|
| hudson.dev.java.net|       2|
|code.msdn.microso...|     172|
|    www.red-gate.com|      15|
|      blogs.msdn.com|     940|
|pandejo.blogspot.com|       2|
|checkstyle.source...|      11|
|  www.say-it-now.com|       1|
|     code.google.com|    3604|
|         osdever.net|       1|
|         us.imdb.com|       3|
|  www.redlizards.com|       1|
|      ant.apache.org|      67|
|         www.98hs.ru|      13|
|services.nuconomy...|       1|
+--------------------+--------+
only showing top 20 rows



Yêu cầu 3 : Tính tổng điểm của User theo từng ngày
+-----------+------------+-----+
|OwnerUserId|CreationDate|Score|
+-----------+------------+-----+
|         26|  2008-08-01|   26|
|         26|  2008-08-01|  144|
|         83|  2008-08-01|   21|
|    	  83|  2008-08-02|   53|
|         26|  2008-08-02|   29|
+-----------+------------+-----+ 

In [78]:
from pyspark.sql import Window
total_score_window = Window.partitionBy("OwnerUserId") \
    .orderBy("CreationDay") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
total_score_df = Questition_converted_schema \
    .select("OwnerUserId",f.to_date("CreationDate").alias("CreationDay"),"Score") \
    .filter(f.expr("OwnerUserId is not null")) \
    .groupBy("CreationDay","OwnerUserId").agg(f.sum("Score").alias("Total Score"))


accumulative_score_df = total_score_df \
    .withColumn("Accumulative score",f.sum(f.col("Total Score")).over(total_score_window))
accumulative_score_df.orderBy("OwnerUserId","CreationDay")

DataFrame[CreationDay: date, OwnerUserId: int, Total Score: bigint, Accumulative score: bigint]

In [77]:
accumulative_score_df.select(f.col("CreationDay"), f.col("OwnerUserId"),f.col("Accumulative score")).show()

+-----------+-----------+------------------+
|CreationDay|OwnerUserId|Accumulative score|
+-----------+-----------+------------------+
| 2008-11-26|          1|                10|
| 2009-01-08|          1|                30|
| 2009-10-08|          1|                58|
| 2009-01-01|          4|                 4|
| 2009-02-14|          4|                13|
| 2010-07-02|          4|                79|
| 2008-12-28|          5|                 0|
| 2009-04-08|          5|                12|
| 2011-03-28|          5|                23|
| 2011-04-06|          5|                25|
| 2012-01-19|          9|                 2|
| 2008-08-05|         17|                14|
| 2010-09-05|         17|                15|
| 2011-01-27|         17|                15|
| 2010-09-22|         20|                 2|
| 2011-04-21|         20|                 4|
| 2011-05-19|         20|                 7|
| 2013-08-02|         20|                 7|
| 2012-04-27|         22|                 1|
| 2008-12-

Yêu cầu 4: Tính tổng số điểm mà User đạt được trong một khoảng thời gian

In [26]:
from pyspark.sql import Window
START = '2008-01-01'
END = '2009-01-01'

Questition_converted_schema.select("OwnerUserId","Score",f.to_date("CreationDate").alias("CreationDay")) \
    .filter((f.col("CreationDay") >= START) & (f.col("CreationDay") <= END)) \
    .groupBy("OwnerUserId") \
    .agg(f.sum("Score")) \
    .orderBy("OwnerUserId").show()

+-----------+----------+
|OwnerUserId|sum(Score)|
+-----------+----------+
|       NULL|      4644|
|          1|        10|
|          4|         4|
|          5|         0|
|         17|        14|
|         23|        27|
|         25|        10|
|         26|        34|
|         27|         9|
|         29|       206|
|         33|       222|
|         35|        25|
|         39|        16|
|         40|         7|
|         41|        16|
|         45|        12|
|         49|        22|
|         51|        30|
|         56|        28|
|         58|       171|
+-----------+----------+
only showing top 20 rows



Yêu cầu 5: Tìm các câu hỏi có nhiều câu trả lời

Đọc dữ liệu từ Answer

In [28]:
Answer_schema = StructType([
    StructField("Body",StringType()),
    StructField("CreationDate",StringType()),
    StructField("Id",IntegerType()),
    StructField("OwnerUserId", StringType()),
    StructField("ParentId",IntegerType()),
    StructField("Score",IntegerType()),
])

Answer_df = spark.read \
    .format("com.mongodb.spark.sql.DefaultSource") \
    .option("uri","mongodb://localhost:27017/ASM1_spark") \
    .option("collection","Answers") \
    .schema(Answer_schema) \
    .load()

Chuẩn hóa dữ liệu trong Awnswer DataFrame và Question DataFrame

In [36]:
Answer_converted_df = Answer_df.withColumn("OwnerUserId",f.expr("case when OwnerUserId = 'NA' then null else OwnerUserId end").cast(IntegerType())) \
    .withColumn("CreationDate",f.col("CreationDate").cast(DateType()))


Answer_converted_df.printSchema()

root
 |-- Body: string (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- Id: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- Score: integer (nullable = true)



In [ ]:
Questition_converted_schema2 = Questition_converted_schema.withColumn("CreationDate",f.col("CreationDate").cast(DateType()))

root
 |-- Body: string (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- ClosedDate: string (nullable = true)
 |-- Id: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Score: integer (nullable = true)



In [40]:
Answer_df.write \
    .bucketBy(3,"ParentId") \
    .mode("overwrite") \
    .saveAsTable("Answer_data")


Questition_converted_schema2.write \
    .bucketBy(3,"Id") \
    .mode("overwrite") \
    .saveAsTable("Question_data")

In [41]:
Answer_converted_df.write \
    .bucketBy(3,"ParentId") \
    .mode("overwrite") \
    .saveAsTable("Answer_converted_data")

In [46]:
df1 = spark.read.table("Question_data")
df2 = spark.read.table("Answer_converted_data")


join_expr = df2.ParentId == df1.Id
join_df = df1.join(df2,join_expr,"left").select(f.expr("Question_data.id as question_id"), f.expr("Answer_converted_data.Id as answerId"))

In [ ]:
join_df.show()

+-----------+--------+
|question_id|answerId|
+-----------+--------+
|         80|     124|
|         80|   10008|
|         80| 3770976|
|         90|      92|
|         90|  202317|
|         90| 1466832|
|        180|     199|
|        180|     529|
|        180|     539|
|        180|   59760|
|        180|   93908|
|        180|  143966|
|        180| 1499720|
|        180| 7815745|
|        180|21634390|
|        470|     473|
|        930|     951|
|        930|    1022|
|        930|     940|
|        930|    2369|
+-----------+--------+
only showing top 20 rows



In [55]:
join_df.groupBy("question_id").agg(f.count("answerId").alias("count")).selectExpr("question_id","case when count > 5 then 1 else 0 end as verify").select(f.sum("verify").alias("Number of good questions")).show()

+------------------------+
|Number of good questions|
+------------------------+
|                   23010|
+------------------------+

